<html>
<head>
<style>
h1 {text-align: center;}
p {text-align: center;}
div {text-align: center;}
</style>
</head>
<body>

<h1>Data Processing</h1>

<h3>Asbjørn Fyhn & Emil Kolko Beckett</h3>
<h3>Seminar: Topics in Sovereign Debt (Spring 2025)</h3>

</body>
</html>

In [13]:
import os
import pandas as pd 
import numpy as np
import warnings
from datetime import datetime
# pretty print
from pprint import pprint

path_to_data = r'/Users/asbjornfyhn/Desktop/Seminar - sovergin debt/Determinants/data/'
path_to_raw_data = path_to_data + 'raw/'

In [ ]:
df_coun_code = pd.read_excel(path_to_raw_data+'EMBI.xlsx',sheet_name='risk_region',)
df_coun_code

,COUNTRY_CODE,RISK_REGION
0,AGO,Africa
1,BEN,Africa
2,CMR,Africa
3,CIV,Africa
4,ETH,Africa
...,...,...
75,BLR,Eastern Europe
76,RUS,Eastern Europe
77,LTU,Eastern Europe
78,SVK,Eastern Europe


## Terms of Trade

In [15]:
df_tot = pd.read_excel(path_to_raw_data+'Terms of trade.xls',skiprows=3,index_col=[0,1])
df_tot = df_tot.iloc[:,2:].stack().reset_index(level=[-1]).rename(columns={'level_2':'Year',0:'ToT_Real'}).set_index('Year',append=True)

# df_com_coun_code = pd.read_excel(path_to_raw_data+'Commodity_Terms_of_Trade_Commodity_.xlsx',sheet_name='country_coding',skiprows=3,index_col=0,usecols='D:E')
# df_tot['Country Code'] = df_tot.index.get_level_values(0).map(df_com_coun_code['Country Code'].to_dict())

df_tot['Date'] = pd.to_datetime((df_tot.index.get_level_values(-1).astype(int)+1).astype(str),format='%Y')-pd.DateOffset(days=1)
df_tot = df_tot.sort_values(by=['Country Code','Year'])


# CALCULATE THE RUNNING VOLATILITY
df_tot = df_tot.reset_index()  # Reset index to avoid duplicate index issue

df_tot['PCT_TOT_Real_1yr'] = df_tot.groupby('Country Code')['ToT_Real'].transform(lambda x: x.pct_change(1))
df_tot['VOL_TOT_Real_1yr'] = df_tot.groupby('Country Code')['PCT_TOT_Real_1yr'].transform(lambda x: x.rolling(10, min_periods=5).std()) * 100

df_tot['PCT_TOT_Real_3yr'] = df_tot.groupby('Country Code')['ToT_Real'].transform(lambda x: x.pct_change(3))
df_tot['VOL_TOT_Real_3yr'] = df_tot.groupby('Country Code')['PCT_TOT_Real_3yr'].transform(lambda x: x.rolling(10, min_periods=5).std()) * 100

df_tot['PCT_TOT_Real_5yr'] = df_tot.groupby('Country Code')['ToT_Real'].transform(lambda x: x.pct_change(5))
df_tot['VOL_TOT_Real_5yr'] = df_tot.groupby('Country Code')['PCT_TOT_Real_5yr'].transform(lambda x: x.rolling(10, min_periods=5).std()) * 100


df_tot = (df_tot
          .set_index(['Country Code','Date']) # Restore the original index if needed
          .drop(columns=['Country Name','Year',]) # Drop the columns that are not needed
          )  
df_tot.index.names = ['COUNTRY_CODE','Date']  # Rename the index
# Display the first 10 rows
df_tot.head(10)

ToT_Real  PCT_TOT_Real_1yr  VOL_TOT_Real_1yr  \
COUNTRY_CODE Date                                                         
ABW          2000-12-31  103.806927               NaN               NaN   
             2001-12-31   96.453092         -0.070841               NaN   
             2002-12-31   92.669278         -0.039230               NaN   
             2003-12-31   99.040741          0.068755               NaN   
             2004-12-31  103.446581          0.044485               NaN   
             2005-12-31  114.376304          0.105656          7.426002   
             2006-12-31  117.517180          0.027461          6.646087   
             2007-12-31  118.482620          0.008215          6.091719   
             2008-12-31  128.506133          0.084599          6.076242   
             2009-12-31   92.695516         -0.278669         11.714761   

                         PCT_TOT_Real_3yr  VOL_TOT_Real_3yr  PCT_TOT_Real_5yr  \
COUNTRY_CODE Date                                                               
ABW          2000-12-31               NaN               NaN               NaN   
             2001-12-31               NaN               NaN               NaN   
             2002-12-31               NaN               NaN               NaN   
             2003-12-31         -0.045914               NaN               NaN   
             2004-12-31          0.072507               NaN               NaN   
             2005-12-31          0.234242               NaN          0.101818   
             2006-12-31          0.186554               NaN          0.218387   
             2007-12-31          0.145351         10.942556          0.278553   
             2008-12-31          0.123538          9.789440          0.297508   
             2009-12-31         -0.211217         15.362150         -0.103929   

                         VOL_TOT_Real_5yr  
COUNTRY_CODE Date                          
ABW          2000-12-31               NaN  
             2001-12-31               NaN  
             2002-12-31               NaN  
             2003-12-31               NaN  
             2004-12-31               NaN  
             2005-12-31               NaN  
             2006-12-31               NaN  
             2007-12-31               NaN  
             2008-12-31               NaN  
             2009-12-31         16.538146

## Debt to GDP and Gold Reserves to GDP

In [16]:
# Load External Debt
df_ex_debt = pd.read_excel(path_to_raw_data+'ext_debt_1.xlsx',)
df_ex_debt = df_ex_debt.set_index(['Country Name','Country Code'],).iloc[:,4:].stack().reset_index(level=[-1]).rename(columns={'level_2':'Year',0:'External Debt'})
df_ex_debt['Year'] = df_ex_debt['Year'].str.extract('(\d+)').astype(int)
df_ex_debt['External Debt'] = df_ex_debt['External Debt'].replace('..',np.nan).astype(float)
df_ex_debt.set_index(['Year'],append=True,inplace=True)

# Load Gold reserves 
df_gold = pd.read_excel(path_to_raw_data+'gold_reserves.xlsx')
df_gold = df_gold.set_index(['Country Name','Country Code'],).iloc[:,2:].stack().reset_index(level=[-1]).rename(columns={'level_2':'Year',0:'Gold Reserves'})
df_gold['Year'] = df_gold['Year'].str.extract('(\d+)').astype(int)
df_gold['Gold Reserves'] = df_gold['Gold Reserves'].replace('..',np.nan).astype(float)
df_gold.set_index(['Year'],append=True,inplace=True)

# Load GDP data
df_gdp = pd.read_excel(path_to_raw_data+'gdp_data.xlsx')
df_gdp = df_gdp.set_index(['Country Name','Country Code'],).iloc[:,2:].stack().reset_index(level=[-1]).rename(columns={'level_2':'Year',0:'GDP'})
df_gdp['Year'] = df_gdp['Year'].str.extract('(\d+)').astype(int)
df_gdp['GDP'] = df_gdp['GDP'].replace('..',np.nan).astype(float)
df_gdp.set_index(['Year'],append=True,inplace=True)

# 
df_coun = pd.concat([df_ex_debt,df_gold,df_gdp],axis=1,ignore_index=False)


df_coun['ex_debt_to_gdp'] = df_coun['External Debt']/df_coun['GDP']
df_coun['gold_to_gdp'] = df_coun['Gold Reserves']/df_coun['GDP']

df_coun['Date'] = pd.to_datetime((df_coun.index.get_level_values(2)+1).astype(str),format='%Y')-pd.DateOffset(days=1)
df_coun = df_coun.reset_index(level=[0, -1], drop=True).set_index('Date',append=True).drop(columns=['External Debt','Gold Reserves','GDP']).sort_index()
df_coun.index.names = ['COUNTRY_CODE','Date']
df_coun.iloc[:5]


ex_debt_to_gdp  gold_to_gdp
COUNTRY_CODE Date                                   
ABW          1960-12-31             NaN          NaN
             1961-12-31             NaN          NaN
             1962-12-31             NaN          NaN
             1963-12-31             NaN          NaN
             1964-12-31             NaN          NaN

### Default 

In [17]:
df_default = pd.read_excel(path_to_raw_data+'Asonuma_Trebesch_DEFAULT_DATABASE.xlsx',sheet_name='DATASET Defaults & Restruct.',usecols='E:H',skiprows=5)
df_default.columns = ['COUNTRY_NAME','COUNTRY_CODE','START_DEFAULT_DATE','END_DEFAULT_DATE']
df_default['START_DEFAULT_DATE'] = pd.to_datetime(df_default['START_DEFAULT_DATE'],format='%Y-%m-%d') 
df_default['START_DEFAULT_YEAR'] = pd.to_datetime((df_default['START_DEFAULT_DATE'].dt.year+1).astype(str),format='%Y')-pd.DateOffset(days=1)
df_default['END_DEFAULT_YEAR'] = pd.to_datetime((df_default['END_DEFAULT_DATE'].dt.year+1).astype(str),format='%Y')-pd.DateOffset(days=1)

# ---------
# Create an empty list to store the expanded rows
expanded_data = []

# Iterate through each row in the original dataframe
for _, row in df_default.iterrows():
    country = row['COUNTRY_CODE']

    # Create range of years from start to end (inclusive)
    years = range(int(row['START_DEFAULT_YEAR'].year), int(row['END_DEFAULT_YEAR'].year) + 1)
    
    # Add each year as a new row with dummy variable = 1
    for year in years:
        expanded_data.append({
            'COUNTRY_CODE': country,
            'Year': year,
            'Default': 1,
            'New_default': 1 if year == row['START_DEFAULT_YEAR'].year else 0
        })

# Convert the list of dictionaries to a dataframe
default_dummy = pd.DataFrame(expanded_data)

default_dummy = default_dummy.sort_values(by=['COUNTRY_CODE','Year','New_default'],).drop_duplicates(subset=['COUNTRY_CODE','Year'],keep='last',)

default_dummy['exit_default'] = 0

for _, row in df_default.iterrows():
    country = row['COUNTRY_CODE']
    year = row['START_DEFAULT_YEAR'].year
    if not df_default.loc[(df_default['COUNTRY_CODE']==country)&(df_default['END_DEFAULT_DATE'].dt.year==year),].empty:
        default_dummy.loc[(default_dummy['COUNTRY_CODE']==country)&(default_dummy['Year']==year),'exit_default'] = 1
        # df_default.loc[(df_default['COUNTRY_CODE']==country)&(df_default['END_DEFAULT_DATE'].dt.year==year),]
    
default_dummy['Date'] = pd.to_datetime((default_dummy['Year']+1).astype(str),format='%Y')-pd.DateOffset(days=1)

# ---------
unique_countries = df_default['COUNTRY_CODE'].unique()
min_year, max_year = df_default['END_DEFAULT_DATE'].dt.year.min(), df_default['END_DEFAULT_DATE'].dt.year.max()
years = range(min_year+1, max_year+1+1)

# Create and
df_ytd = pd.DataFrame(index=pd.MultiIndex.from_product([unique_countries, pd.to_datetime(years,format='%Y')-pd.DateOffset(days=1)], names=['COUNTRY_CODE', 'Date']), columns=['YTD'])
df_ytd['YTD'] = -1

for coun in unique_countries[:]: # loop over unique countries
    temp_coun = default_dummy.loc[(default_dummy['COUNTRY_CODE']==coun),:]
    for i, year in enumerate(years): # loop over years
        if temp_coun.loc[(temp_coun['Year']<=year),:].empty:
            df_ytd.loc[(coun, pd.to_datetime(year+1,format='%Y')-pd.DateOffset(days=1)), 'YTD'] = 11
        else:
            if not temp_coun.loc[(temp_coun['Year']==year),:].empty:
                df_ytd.loc[(coun, pd.to_datetime(year+1,format='%Y')-pd.DateOffset(days=1)), 'YTD'] = 0
            else:
                df_ytd.loc[(coun, pd.to_datetime(year+1,format='%Y')-pd.DateOffset(days=1)), 'YTD'] = year - temp_coun.loc[(temp_coun['Year']<=year),'Year'].max()

In [18]:
df_default_all = pd.merge(df_ytd, default_dummy.set_index(['COUNTRY_CODE','Date']).drop(columns=['Year']),left_index=True,right_index=True,how='outer')
df_default_all[['Default','New_default','exit_default']] = df_default_all[['Default','New_default','exit_default']].fillna(0)
df_default_all = df_default_all.loc[df_default_all['YTD']!=-1]

# Lag the YTD variable
df_default_all['YTD'] = df_default_all.groupby('COUNTRY_CODE')['YTD'].shift(1)

df_default_all.loc[df_default_all.index.get_level_values(0)=='ARG'].iloc[:]

YTD  Default  New_default  exit_default
COUNTRY_CODE Date                                                
ARG          1979-12-31   NaN      0.0          0.0           0.0
             1980-12-31  11.0      0.0          0.0           0.0
             1981-12-31  11.0      0.0          0.0           0.0
             1982-12-31  11.0      1.0          1.0           0.0
             1983-12-31   0.0      1.0          0.0           0.0
             1984-12-31   0.0      1.0          0.0           0.0
             1985-12-31   0.0      1.0          1.0           1.0
             1986-12-31   0.0      1.0          0.0           0.0
             1987-12-31   0.0      1.0          0.0           0.0
             1988-12-31   0.0      1.0          1.0           0.0
             1989-12-31   0.0      1.0          0.0           0.0
             1990-12-31   0.0      1.0          0.0           0.0
             1991-12-31   0.0      1.0          0.0           0.0
             1992-12-31   0.0      1.0          0.0           0.0
             1993-12-31   0.0      1.0          0.0           0.0
             1994-12-31   0.0      0.0          0.0           0.0
             1995-12-31   1.0      0.0          0.0           0.0
             1996-12-31   2.0      0.0          0.0           0.0
             1997-12-31   3.0      0.0          0.0           0.0
             1998-12-31   4.0      0.0          0.0           0.0
             1999-12-31   5.0      0.0          0.0           0.0
             2000-12-31   6.0      0.0          0.0           0.0
             2001-12-31   7.0      1.0          1.0           0.0
             2002-12-31   0.0      1.0          0.0           0.0
             2003-12-31   0.0      1.0          0.0           0.0
             2004-12-31   0.0      1.0          0.0           0.0
             2005-12-31   0.0      1.0          0.0           0.0
             2006-12-31   0.0      0.0          0.0           0.0
             2007-12-31   1.0      0.0          0.0           0.0
             2008-12-31   2.0      0.0          0.0           0.0
             2009-12-31   3.0      0.0          0.0           0.0
             2010-12-31   4.0      0.0          0.0           0.0
             2011-12-31   5.0      0.0          0.0           0.0
             2012-12-31   6.0      0.0          0.0           0.0
             2013-12-31   7.0      0.0          0.0           0.0
             2014-12-31   8.0      0.0          0.0           0.0
             2015-12-31   9.0      0.0          0.0           0.0
             2016-12-31  10.0      0.0          0.0           0.0
             2017-12-31  11.0      0.0          0.0           0.0
             2018-12-31  12.0      0.0          0.0           0.0
             2019-12-31  13.0      1.0          1.0           0.0
             2020-12-31   0.0      1.0          0.0           0.0
             2021-12-31   0.0      0.0          0.0           0.0

## Merging data together


In [29]:
df = df_tot.merge(df_default_all,left_index=True,right_index=True,how='left')
df = df.merge(df_coun, left_index=True, right_index=True, how='left')
# Merge risk region on
df = df.reset_index(level=1).merge(df_coun_code.set_index('COUNTRY_CODE'),left_index=True,right_index=True,how='left').set_index('Date',append=True)
# df['Default'] = df['Default'].fillna(0)
print(f'Shape of data {df.shape[0]}')
df.loc[df.index.get_level_values(0)=='ARG'].iloc[:,-10:]



Shape of data 6026


VOL_TOT_Real_3yr  PCT_TOT_Real_5yr  VOL_TOT_Real_5yr  \
COUNTRY_CODE Date                                                               
ARG          1980-12-31               NaN               NaN               NaN   
             1981-12-31               NaN               NaN               NaN   
             1982-12-31               NaN               NaN               NaN   
             1983-12-31               NaN               NaN               NaN   
             1984-12-31               NaN               NaN               NaN   
             1985-12-31               NaN         -0.114187               NaN   
             1986-12-31               NaN          0.051734               NaN   
             1987-12-31         14.981097          0.039313               NaN   
             1988-12-31         15.818988         -0.106164               NaN   
             1989-12-31         16.511006         -0.174747         10.072052   
             1990-12-31         16.359179         -0.324682         14.042939   
             1991-12-31         18.466235          0.008487         13.515447   
             1992-12-31         21.480201          0.134963         14.800382   
             1993-12-31         25.656201          0.281046         17.927823   
             1994-12-31         25.423066          0.407533         21.697678   
             1995-12-31         25.209630          0.441120         24.762073   
             1996-12-31         25.179522          0.086022         24.748689   
             1997-12-31         25.086372          0.072165         24.711504   
             1998-12-31         23.461928         -0.009223         24.070667   
             1999-12-31         21.563636         -0.064245         22.935546   
             2000-12-31         18.867026          0.091743         17.417864   
             2001-12-31         18.570351         -0.016191         17.648660   
             2002-12-31         16.296344         -0.002554         18.239720   
             2003-12-31          5.351678          0.139507         17.458987   
             2004-12-31          7.169171          0.273683         15.349183   
             2005-12-31          7.336534          0.123206          9.824613   
             2006-12-31          7.370818          0.192703         10.575532   
             2007-12-31          7.307479          0.246878         11.802282   
             2008-12-31          9.176235          0.286241         12.509459   
             2009-12-31          7.534486          0.213089         10.746020   
             2010-12-31          6.603212          0.295604         11.333849   
             2011-12-31          5.712171          0.368383         10.619528   
             2012-12-31          5.669286          0.343798          8.143058   
             2013-12-31          5.793455          0.121677          8.422477   
             2014-12-31          8.291772          0.090210          9.671000   
             2015-12-31         11.466389          0.005644         11.609184   
             2016-12-31         11.974304         -0.033133         14.059837   
             2017-12-31         12.458175         -0.098018         16.612685   
             2018-12-31         11.158060         -0.025209         16.886665   
             2019-12-31         10.365251         -0.013443         17.138990   
             2020-12-31          9.330908          0.039752         15.847674   
             2021-12-31          8.610244          0.073687         12.211316   

                          YTD  Default  New_default  exit_default  \
COUNTRY_CODE Date                                                   
ARG          1980-12-31  11.0      0.0          0.0           0.0   
             1981-12-31  11.0      0.0          0.0           0.0   
             1982-12-31  11.0      1.0          1.0           0.0   
             1983-12-31   0.0      1.0          0.0           0.0   
             1984-12-31   0.0      1.0        

In [ ]:

df.to_excel(path_to_data+'prob_model_data.xlsx')

## Notes for Asbjørn 
- Estimér en logit model. 
    - Rapport resultaterne. 
    - Beregn average partial effect 
    - Bootstrap SD på APE
- Predicted spreads: 
    - Beregn som i en binomial model: $S_{up}$ (value 1) og $S_{down}$ (recovery rate)
- Years since default: 
    - skal en observation pilles ud hvis YTD=0? 
    - $Y_{i, t+1} =1$ hvis landet defaulter på ny i år $t+1$ .

